In [16]:
import configparser

config = configparser.RawConfigParser()
config.read(filenames="twitter_keys.txt")


accesstoken = config.get("twitter", "accesstoken")
accesstokensecret = config.get("twitter", "accesstokensecret")
apikey = config.get("twitter", "apikey")
apisecretkey = config.get("twitter", "apisecretkey")

In [18]:
import tweepy as tw
auth = tw.OAuthHandler(apikey,apisecretkey)
auth.set_access_token(accesstoken,accesstokensecret)
api = tw.API(auth,wait_on_rate_limit=True)


In [19]:
def clean_tweet_text(tweet):
    """Clean tweet text by removing links, special characters, and extra whitespace."""
    # Remove links
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet)
    # Remove special characters and extra whitespace
    tweet = re.sub(r'[^\w\s]', '', tweet)
    tweet = re.sub(r'\s+', ' ', tweet).strip()
    return tweet

def scrape_tweets(api, ticker, num_tweets):
    tweets = []
    for tweet in tw.Cursor(api.search_tweets, q=ticker, lang='en', tweet_mode='extended').items(num_tweets):
        # Exclude tweets with Discord or chatroom links
        if 'discord.gg' not in tweet.full_text and 'chatroom' not in tweet.full_text:
            # Exclude tweets mentioning other tickers in their content
            if all(t not in tweet.full_text for t in ticker.split()):
                cleaned_text = clean_tweet_text(tweet.full_text)
                tweets.append(cleaned_text)
    return tweets

In [20]:
# Specify the ticker symbol and the number of tweets to scrape
ticker = 'AAPL'
num_tweets = 1

In [21]:
# Scrape relevant tweets
relevant_tweets = scrape_tweets(api, ticker, num_tweets)

# Print the scraped tweets
for tweet in relevant_tweets:
    print(tweet)


Forbidden: 403 Forbidden
453 - You currently have access to Twitter API v2 endpoints and limited v1.1 endpoints only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-leve

In [24]:
search_word = '#AAPL'
date_since = '2020-05-28'

tweets = tw.Cursor(api.search,q = search_word, lang ='en',since = date_since).items(1)
tweet_details = [[tweet.geo,tweet.text,tweet.user.screen_name,tweet.user.location]for tweet in tweets]



AttributeError: 'API' object has no attribute 'search'

In [3]:
import tweepy
import pandas as pd

# Create a list of ticker symbols
ticker_symbols = ["AAPL", "GOOG", "MSFT"]

# Create a Twitter API object
auth = tweepy.OAuthHandler("CONSUMER_KEY", "CONSUMER_SECRET")
auth.set_access_token("ACCESS_TOKEN", "ACCESS_TOKEN_SECRET")
api = tweepy.API(auth)

# Search for tweets that contain each ticker symbol
tweets = []
for ticker_symbol in ticker_symbols:
    tweets.extend(api.search(q=ticker_symbol))

# Save the tweets to a CSV file
df = pd.DataFrame(tweets)
df.to_csv("tweets.csv", index=False)


ModuleNotFoundError: No module named 'tweepy'

In [1]:
import twint
import pandas as pd
import re

def get_stock_tweets(stock_ticker, num_tweets):
    # Configuring twint search
    c = twint.Config()
    c.Search = stock_ticker
    c.Lang = 'en'
    c.Limit = num_tweets
    c.Pandas = True

    # Running search
    twint.run.Search(c)

    # Getting a pandas DataFrame of tweets
    tweets_df = twint.storage.panda.Tweets_df
    
    return tweets_df

def filter_tweets(df, stock_ticker):
    # Define other tickers pattern
    other_tickers = re.compile(r'\$[a-zA-z]+')

    # Exclude tweets with other tickers, links, and possible chatroom messages
    df_filtered = df[~df['tweet'].str.contains('|'.join(['http', 'https', 'discord', 'chat', 'room']))]
    df_filtered = df[df['tweet'].apply(lambda x: len(re.findall(other_tickers, x)) <= 1)]

    return df_filtered

def store_tweets(df, stock_ticker):
    # Store the data to csv
    df.to_csv(f'{stock_ticker}_tweets.csv', index=False)

# Use the functions
stock_ticker = '$AAPL' # or any ticker
num_tweets = 1000 # or any number of tweets
df = get_stock_tweets(stock_ticker, num_tweets)
df_filtered = filter_tweets(df, stock_ticker)
store_tweets(df_filtered, stock_ticker)


RuntimeError: This event loop is already running